In [ ]:
### Model Dict
modelDict = {}
df_copy = df.copy()
df_cancer_copy = df_cancer.copy()
for i in range(10):
    # Drop first row
    df_copy.drop(index=df_copy.index[0], 
            axis=0, 
            inplace=True)
    X = df_copy
#     X = X.reset_index().drop(columns = ["index","Year"])
    X = X.reset_index().drop(columns = ["index","Year","Time"])

    
    # Drop last row
    df_cancer_copy.drop(index=df_cancer_copy.index[-1], 
            axis=0, 
            inplace=True)

    y = df_cancer_copy['Cancer incidence']
    y = y.reset_index().drop(columns = "index")
    y = y.astype(float)

    # Define the model
    X = sm.add_constant(X)
    model = sm.OLS(y, X)

    # Fit the model
    model_fit = model.fit()

    # Print the model summary
#     print("======================================")
#     print("Model: " + str(i))
#     print(model_fit.summary())
    
    # Add to dict
    modelDict["model"+str(i)] = {}
    modelDict["model"+str(i)]["_rsquared"]= model_fit.rsquared
    modelDict["model"+str(i)]["_rsquared_adj"]= model_fit.rsquared_adj
    modelDict["model"+str(i)]["_pvalues"]= model_fit.pvalues
    modelDict["model"+str(i)]["_params"]= model_fit.params

In [1]:
### Create Graph
df_copy = df.copy()
df_copy = df_copy.set_index("Year")
df_combined = pd.concat([df_copy, df_cancer], axis = 1)

df_copy.index.rename('', inplace=True)

# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(df_combined.index.astype(str), df_combined['Yearly Microplastics Emission'], color="red", marker="o")
# set x-axis label
ax.set_xlabel("Year",fontsize=14)
x = df.index.tolist()
ax.set_xticklabels(x,rotation=90)
# set y-axis label
ax.set_ylabel("MPP",color="red",fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(df_combined.index.astype(str), df_combined['Cancer incidence'],color="blue",marker="o")
ax2.set_ylabel("Cancer",color="blue",fontsize=14)
x = df_combined.index.tolist()
ax2.set_xlabel("Year",fontsize=14)
ax2.set_xticklabels(x,rotation=90)
plt.show()
# save the plot as a file
fig.savefig('mpp_w_cancer.jpg',
            format='jpeg',
            dpi=100,
            bbox_inches='tight')

NameError: name 'df' is not defined

In [ ]:
### cleaning up cancer data
df_cancer = pd.read_csv('original-datasets/old-cancer.csv')
df_cancer = df_cancer.T
df_cancer.columns = df_cancer.iloc[0] 
df_cancer = df_cancer[1:]
df_cancer = df_cancer.replace("-", 0)
df_cancer.head()

cancer_columns = list(df_cancer.columns)
cancer_countries = []
for i in cancer_columns:
    country_name = i.split(",")[0]
    if country_name not in cancer_countries:
        cancer_countries.append(country_name)

cancer = pd.DataFrame()

for column in df_cancer:
    df_cancer[column] = df_cancer[column].astype(int)
    column_name = column.split(",")[0]
    if column_name not in list(cancer.columns):
        cancer[column_name] = df_cancer[column]
cancer["USA"] = cancer["USA"] + cancer["USA: White"] + cancer["USA: Black"]
cancer = cancer.drop(["USA: White", "USA: Black"], axis = 1)
cancer.insert(0, "Year", list(df_cancer.index))
cancer

cancer.to_csv("cancer.csv", index = False)

In [ ]:
### populate median age data
age_model_df = pd.read_csv('original-datasets/age.csv')
age_model_df = age_model_df.groupby("Year").mean()
age_model_df = age_model_df.reset_index()
age_model_df

Y_age = age_model_df["MedianAge"]
Y_age = age_model_df.astype(float)
X_age = age_model_df["Year"]

X_age = sm.add_constant(X_age)
model_age = sm.OLS(Y_age, X_age)

# Fit the model
model_age_fit = model_age.fit()

# Print the model summary
print(model_age_fit.summary())

age_df = pd.DataFrame()
age_df.insert(0, 'Year', range(1970, 2022))
age_df["MedianAge"] = model_age_fit.predict(sm.add_constant(age_df["Year"]))
age_df.to_csv("age.csv")

In [ ]:
### BMI merging
male_df = pd.read_csv('bmi/male-bmi.csv')
male_df = male_df.loc[(male_df['Year'] >= 1975) & (male_df['Year'] <= 2000)]
male_df = male_df.groupby(['Year']).median()
female_df = pd.read_csv('bmi/female-bmi.csv')
female_df = female_df.loc[(female_df['Year'] >= 1975) & (female_df['Year'] <= 2000)]
female_df = female_df.groupby(['Year']).median()
gender_df = pd.read_excel('bmi/gender-ratio.xlsx')
gender_df["Male_Per"] = gender_df["Male"]/(gender_df["Male"] + gender_df["Female"])
gender_df["Female_Per"] = 1 - gender_df["Male_Per"]
gender_df = gender_df.loc[(gender_df['Year'] >= 1975) & (gender_df['Year'] <= 2000)]
male_df = male_df.reset_index()
female_df = female_df.reset_index()
gender_df = gender_df.reset_index()
bmi_df = pd.DataFrame()
bmi_df["Year"] = gender_df["Year"]
bmi_df["BMI"] = male_df["Mean BMI (male)"]*gender_df["Male_Per"] + female_df["Mean BMI (female)"]*gender_df["Female_Per"]

bmi_df.head()